## Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=2

In [ ]:
import torch

In [ ]:
%run -n ../train_classification.py

In [ ]:
DEVICE = torch.device('cuda')
DEVICE

## Train

In [ ]:
max_samples = 12000
n_epochs = 3

In [ ]:
%%time

run_id, compiled_model, train_metrics, val_metrics = train_from_scratch(
    get_timestamp(),
    'imagenet',
    shuffle=True,
    cnn_name='tiny-densenet',
    max_samples=max_samples,
    image_size=128,
    batch_size=1200,
    norm_by_sample=True,
    n_epochs=n_epochs,
    early_stopping=False,
    lr_sch_metric=None,
    debug=True,
    num_workers=0,
    device='cuda'
)

In [ ]:
print(run_id)